In [ ]:

import yaml
import matplotlib.pyplot as plt
# from train_eval.trainer import Trainer
import torch
import torch.nn as nn
import os
import numpy as np
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = "4"
os.environ["MKL_NUM_THREADS"] = "16"
os.environ["NUMEXPR_NUM_THREADS"] = "16"
os.environ["OMP_NUM_THREADS"] = "16"

In [ ]:
with open("configs/occlusion_train_v2_vis.yml", 'r') as yaml_file:
    cfg = yaml.safe_load(yaml_file)

In [ ]:
from train_eval.evaluator import Evaluator
trainer = Evaluator(cfg, "/home/stanliu/data/mnt/nuScenes/nuscenes", "vis_data","track_completion_model/track_completion_att2.tar")

In [ ]:
import torch.optim
from typing import Dict, Union
import torch
import numpy as np
import os
from datasets.nuScenes.prediction import PredictHelper_occ
from nuscenes.prediction.input_representation.static_layers import *
from nuscenes.prediction.input_representation.combinators import Rasterizer
from pyquaternion import Quaternion
from nuscenes.eval.common.utils import quaternion_yaw
import logging
from numpy import linalg as LA
import matplotlib.pyplot as plt
import matplotlib.image as im
from PIL import Image
import copy
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from executables.track_completion import *
def get_cam(sample_record,nusc,anntoken):
    cams = [key for key in sample_record['data'].keys() if 'CAM' in key]
    for cam in cams:
        _, boxes, _ = nusc.get_sample_data(sample_record['data'][cam], box_vis_level=BoxVisibility.ANY,
                                                selected_anntokens=[anntoken])
        if len(boxes) > 0:
            break  # We found an image that matches. Let's abort.
    assert len(boxes) > 0, 'Error: Could not find image where annotation is visible. ' \
                            'Try using e.g. BoxVisibility.ANY.'
    assert len(boxes) < 2, 'Error: Found multiple annotations. Something is wrong!'

    cam = sample_record['data'][cam]
    return cam
def render_occ_anns(frame,nusc: NuScenes,global_coord,global_rotation,save_dir,key,frame_id,save):
    anntoken=frame['ann_token']
    ann_record = nusc.get('sample_annotation', anntoken)
    sample_record = nusc.get('sample', ann_record['sample_token'])
    cam = get_cam(sample_record,nusc,anntoken)
    z=nusc.get('sample_annotation', anntoken)['translation'][-1]
    data_path, boxes, camera_intrinsic = nusc.get_sample_data(cam, selected_anntokens=[anntoken])
    sd_record = nusc.get('sample_data', cam)
    cs_record = nusc.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
    # sensor_record = nusc.get('sensor', cs_record['sensor_token'])
    pose_record = nusc.get('ego_pose', sd_record['ego_pose_token'])

    im = Image.open(data_path)
    fig, axes = plt.subplots(1, 1, figsize=(9, 9))
    axes.imshow(im)
    axes.set_title(nusc.get('sample_data', cam)['channel'])
    axes.axis('off')
    axes.set_aspect('equal')
    def get_color(name):
        return nusc.colormap[name]
    assert len(boxes)==1
    pred_box=copy.deepcopy(boxes[0])
    pred_box.name="human.pedestrian.adult"
    pred_box.center=global_coord+[z]
    pred_box.orientation=Quaternion(global_rotation)

    pred_box.translate(-np.array(pose_record['translation']))
    pred_box.rotate(Quaternion(pose_record['rotation']).inverse)

    #  Move box to sensor coord system.
    pred_box.translate(-np.array(cs_record['translation']))
    pred_box.rotate(Quaternion(cs_record['rotation']).inverse)
    boxes.append(pred_box)
    # boxes.append()
    for box in boxes:
        c = np.array(get_color(box.name)) / 255.0
        box.render(axes, view=camera_intrinsic, normalize=True, colors=(c, c, c))
    if save:
        out_dir=os.path.join(save_dir,key)
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        out_path=os.path.join(out_dir,str(frame_id))
        plt.savefig(out_path)
        plt.close(fig)
    return

@torch.no_grad()
def local_pose_to_image(local_poses,pose_mask,resolution,img_size,arror_length=None):
    '''local_poses: [T,4] 
    mask: [T]
    '''
    if arror_length is None:
        arror_length=8
    y_m=np.asarray(local_poses[:,1][pose_mask].cpu())
    x_m=np.asarray(local_poses[:,0][pose_mask].cpu())
    img_origin=np.round(np.asarray(img_size)/2).astype(np.int)
    x=img_origin[1]+x_m*resolution
    y=img_origin[0]-y_m*resolution
    yaw=np.asarray((local_poses[:,2][pose_mask]).cpu())
    dy=-np.sin(yaw+np.pi/2)*arror_length
    dx=np.cos(yaw+np.pi/2)*arror_length
    return x,y,dx,dy

@torch.no_grad()
def visualize(inputs: Dict,ground_truth: Dict,predictions: Dict,helper: PredictHelper_occ, selector,token_dicts,save_folder='./tmp', mode='refine',save=False):

    upper_limit=100
    batch_size=len(predictions['traj'])
    layer_names = ['drivable_area', 'ped_crossing']
    maps= load_all_maps(helper)
    colors = [(255, 255, 255), (119, 136, 153)]
    for sample_id in range(batch_size):
        if sample_id>upper_limit:
            return
        # try:
        if not selector[sample_id]:
            continue
        instance_token=inputs['instance_token'][sample_id]
        sample_token=inputs['sample_token'][sample_id]
        future=inputs['target_agent_representation']['future']['traj'][sample_id]
        mask_fut=inputs['target_agent_representation']['future']['mask'][sample_id]
        hist=inputs['target_agent_representation']['history']['traj'][sample_id]
        nearest_idx=np.where(mask_fut[:, 0].cpu() == 0)[0][-1]
        prediction_horizon=future[nearest_idx,-1]
        sample_annotation = helper.get_sample_annotation(instance_token, sample_token)
            
        map_name = helper.get_map_name_from_sample_token(sample_token)
        x, y = sample_annotation['translation'][:2]
        yaw = quaternion_yaw(Quaternion(sample_annotation['rotation']))
        yaw_corrected = correct_yaw(yaw)
        global_pose=(x,y,yaw_corrected)
        if 'origin' in inputs:
            origin=tuple([inputs['origin'][sample_id,0].item(),inputs['origin'][sample_id,1].item(),inputs['origin'][sample_id,2].item()])
        else:
            coords_fut,global_yaw_fut,time_fut = helper.get_future_for_agent(instance_token, sample_token, seconds=2+prediction_horizon, in_agent_frame=False,add_yaw_and_time=True)

            sep_idx= np.searchsorted(time_fut, (prediction_horizon-0.001).item())
            origin_fut=coords_fut[sep_idx][0],coords_fut[sep_idx][1],correct_yaw(quaternion_yaw(Quaternion(global_yaw_fut[sep_idx])))
            origin=tuple((np.asarray(global_pose)+np.asarray(origin_fut))/2)
        dist=LA.norm(future[0,:2].cpu(),ord=2)
        image_side_length = 2 * max(25,dist+10)
        image_side_length_pixels = 400
        resolution=image_side_length_pixels/image_side_length
        patchbox = get_patchbox(origin[0], origin[1], image_side_length)

        angle_in_degrees = angle_of_rotation(origin[2]) * 180 / np.pi

        canvas_size = (image_side_length_pixels, image_side_length_pixels)
        masks = maps[map_name].get_map_mask(patchbox, angle_in_degrees, layer_names, canvas_size=canvas_size)
        
        images = []
        for mask, color in zip(masks, colors):
            images.append(change_color_of_binary_mask(np.repeat(mask[::-1, :, np.newaxis], 3, 2), color))
        if mode=='refine':
            traj = predictions['refined_traj'][sample_id].squeeze(0)
            yaw = predictions['refined_yaw'][sample_id]
        elif mode=='raw':
            traj = predictions['traj'][sample_id].squeeze(0)
            yaw = predictions['yaw'][sample_id]
        lanes=inputs['map_representation']['lane_node_feats'][sample_id].flatten(0,1).clone()
        lanes_mask=inputs['map_representation']['lane_node_feats'][sample_id].flatten(0,1)[:,0].bool()
        pred = torch.cat((traj,yaw),-1)
        pose_pred_mask=~(predictions['mask'][sample_id]).bool()
        gt = ground_truth['traj'][sample_id]
        image = Rasterizer().combine(images)
        pose_future_mask=~inputs['target_agent_representation']['future']['mask'][sample_id][:,0].bool()
        pose_hist_mask=~inputs['target_agent_representation']['history']['mask'][sample_id][:,0].bool()
        xs, ys, dxs, dys=local_pose_to_image(future,pose_future_mask,resolution,canvas_size)
        xsh, ysh, dxsh, dysh=local_pose_to_image(hist,pose_hist_mask,resolution,canvas_size)
        xsp, ysp, dxsp, dysp=local_pose_to_image(pred,pose_pred_mask,resolution,canvas_size,5)
        xsg, ysg, dxsg, dysg=local_pose_to_image(gt,pose_pred_mask,resolution,canvas_size,5)
        xsl, ysl, dxsl, dysl=local_pose_to_image(lanes,lanes_mask,resolution,canvas_size,2.5)
        # plt.imshow(image)
        fig = plt.figure(figsize = (8,8))
        ax = fig.add_subplot(1,1,1) 
        
        for x, y, dx, dy in zip(xs, ys, dxs, dys):
            ax.arrow(x, y, dx, dy, width=0.8, color=(1,0,0,1))
        for x, y, dx, dy in zip(xsh, ysh, dxsh, dysh):
            ax.arrow(x, y, dx, dy, width=0.8, color=(0,1,0,1))
        for x, y, dx, dy in zip(xsp, ysp, dxsp, dysp):
            ax.arrow(x, y, dx, dy, width=1.0, color=(0.0,0,1,1))
        for x, y, dx, dy in zip(xsg, ysg, dxsg, dysg):
            ax.arrow(x, y, dx, dy, width=1.0, color=(1,0,1,0.3))
        for x, y, dx, dy in zip(xsl, ysl, dxsl, dysl):
            ax.arrow(x, y, dx, dy, width=0.5, color=(1,0.5,0,0.3))
        ax.imshow(image)
        ax.grid(False)
        fig.canvas.draw()
        image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        # plt.close(fig)
        if save:
            plt.close(fig)
            out_dir=os.path.join(save_folder,instance_token+'_'+sample_token)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
            im.imsave(os.path.join(out_dir,'bev'), image_from_plot)
            # except:
            #     continue
        # fig, axes = plt.subplots(1, 2, figsize=(18, 9))

        key=instance_token+"_"+sample_token

        missing_frames=token_dicts[key]
        for idx,frame in enumerate(missing_frames):
            yaw=pred[idx,-1].item()

            global_coord=local_to_global(origin, tuple(pred[idx,:2].cpu().numpy()))
            global_rotation=get_global_rotation(origin,yaw)

            render_occ_anns(frame,helper.data,list(global_coord),global_rotation,save_folder,key,idx,save)
        # break
    return
@torch.no_grad()
def visualize_preparations(vis_dict, inputs: Dict,ground_truth: Dict,predictions: Dict,helper: PredictHelper_occ, selector,token_dicts,save_folder='./tmp', mode='refine',save=False):

    batch_size=len(predictions['traj'])
    for sample_id in range(batch_size):
        if not selector[sample_id]:
            continue
        instance_token=inputs['instance_token'][sample_id]
        sample_token=inputs['sample_token'][sample_id]
        key=instance_token+"_"+sample_token
        
        future=inputs['target_agent_representation']['future']['traj'][sample_id]
        mask_fut=inputs['target_agent_representation']['future']['mask'][sample_id]
        hist=inputs['target_agent_representation']['history']['traj'][sample_id]
        vis_dict[key]={
            "future":future,
            "mask_fut":mask_fut,
            "history":hist,
            "origin":inputs['origin'][sample_id],
            "traj" :predictions['refined_traj'][sample_id],
            "yaw" :predictions['refined_yaw'][sample_id],
            "pose_pred_mask":predictions['mask'][sample_id],
            "gt" : ground_truth['traj'][sample_id],
            "lanes":inputs['map_representation']['lane_node_feats'][sample_id],
            "lanes_mask":inputs['map_representation']['lane_node_masks'][sample_id],
            "pose_future_mask":inputs['target_agent_representation']['future']['mask'][sample_id],
            "pose_hist_mask":inputs['target_agent_representation']['history']['mask'][sample_id]
        }

    return vis_dict

In [ ]:
tokens_list=trainer.dl.dataset.data_list
token_dicts={}
full_token_dicts={}
for token_dict in tokens_list:
    key=token_dict['start']['ins_token']+"_"+token_dict['start']['sample_token']
    token_dicts[key]=token_dict['missing_frames']
    full_token_dicts[key]=token_dict

In [ ]:
import train_eval.utils as u
helper= PredictHelper_occ(trainer.dl.dataset.helper.data)
vis_dicts={}
with torch.no_grad():
    for i,data in enumerate(trainer.dl):
        print(i,len(trainer.dl))
        # torch.cuda.empty_cache()
        # Load data
        data = u.send_to_device(u.convert_double_to_float(data))
        data_test=data['inputs']
        gt_test=data['ground_truth']
        
        selectors=(torch.norm(gt_test['traj'][:,:,-1],1,dim=1).cpu()>1.5)*(torch.sum(torch.norm(gt_test['traj'][:,:,:2],2,dim=-1),dim=-1).cpu()>10.0)
        predcitions=trainer.model(data_test)
        # visualize(data_test,gt_test,predcitions,helper, selectors, token_dicts,save=True)
        vis_dicts=visualize_preparations(vis_dicts, data_test, gt_test, predcitions, helper, selectors, token_dicts)
        # break

In [ ]:
vis_sample_token="d45c8e8a7e9e45458003153e96fa3259_f9475d1f1f96401c81ce511a93578a96"
vis_dict=vis_dicts[vis_sample_token]
missing_frames=token_dicts[vis_sample_token]
token_dict=full_token_dicts[vis_sample_token]
hist_frames=3
fut_frames=3

In [ ]:

layer_names = ['drivable_area', 'ped_crossing']
maps= load_all_maps(helper)
colors = [(255, 255, 255), (119, 136, 153)]
instance_token,sample_token=vis_sample_token.split("_")
start_ann_token=token_dict['start']['ann_token']
hist_tokens=[start_ann_token]
end_ann_token=token_dict['end']['ann_token']
fut_tokens=[end_ann_token]
metadata=helper.data.get('sample_annotation',start_ann_token)
for idx in range(hist_frames-1):
    if metadata['prev'] == '':
        break
    hist_token=metadata['prev']
    hist_tokens.append(hist_token)
    metadata=helper.data.get('sample_annotation',hist_token)
# hist_tokens=hist_tokens
metadata=helper.data.get('sample_annotation',end_ann_token)
for idx in range(fut_frames-1):
    if metadata['next'] == '':
        break
    fut_token=metadata['next']
    fut_tokens.append(fut_token)
    metadata=helper.data.get('sample_annotation',fut_token)
# fut_tokens=fut_tokens[::-1]
future=vis_dict['future']
mask_fut=vis_dict['mask_fut']
hist=vis_dict['history']
nearest_idx=np.where(mask_fut[:, 0].cpu() == 0)[0][-1]
prediction_horizon=future[nearest_idx,-1]
sample_annotation = helper.get_sample_annotation(instance_token, sample_token)
    
map_name = helper.get_map_name_from_sample_token(sample_token)
x, y = sample_annotation['translation'][:2]
yaw = quaternion_yaw(Quaternion(sample_annotation['rotation']))
yaw_corrected = correct_yaw(yaw)
global_pose=(x,y,yaw_corrected)
origin=tuple([vis_dict['origin'][0].item(),vis_dict['origin'][1].item(),vis_dict['origin'][2].item()])
dist=LA.norm(future[0,:2].cpu(),ord=2)
image_side_length = 2 * max(25,dist+10)
image_side_length_pixels = 1200
resolution=image_side_length_pixels/image_side_length
patchbox = get_patchbox(origin[0], origin[1], image_side_length)

angle_in_degrees = angle_of_rotation(origin[2]) * 180 / np.pi

canvas_size = (image_side_length_pixels, image_side_length_pixels)
masks = maps[map_name].get_map_mask(patchbox, angle_in_degrees, layer_names, canvas_size=canvas_size)

images = []
for mask, color in zip(masks, colors):
    images.append(change_color_of_binary_mask(np.repeat(mask[::-1, :, np.newaxis], 3, 2), color))

traj = vis_dict['traj'].squeeze(0)
yaw = vis_dict['yaw']
lanes=vis_dict['lanes'].flatten(0,1).clone()
lanes_mask=~vis_dict['lanes_mask'].flatten(0,1)[:,0].bool()
pred = torch.cat((traj,yaw),-1)
pose_pred_mask=~(vis_dict["pose_pred_mask"]).bool()
gt = vis_dict["gt"]
image = Rasterizer().combine(images)
pose_future_mask=~vis_dict["pose_future_mask"][:,0].bool()
pose_hist_mask=~vis_dict["pose_hist_mask"][:,0].bool()
xs, ys, dxs, dys=local_pose_to_image(future,pose_future_mask,resolution,canvas_size,30)
xsh, ysh, dxsh, dysh=local_pose_to_image(hist,pose_hist_mask,resolution,canvas_size,30)
xsp, ysp, dxsp, dysp=local_pose_to_image(pred,pose_pred_mask,resolution,canvas_size,30)
xsg, ysg, dxsg, dysg=local_pose_to_image(gt,pose_pred_mask,resolution,canvas_size,30)
xsl, ysl, dxsl, dysl=local_pose_to_image(lanes,lanes_mask,resolution,canvas_size,8)

In [ ]:
xshr=xsh[::-1]
yshr=ysh[::-1]
dxshr=dxsh[::-1]
dyshr=dysh[::-1]
# xshr=xsh
# yshr=ysh
# dxshr=dxsh
# dyshr=dysh
hist_imgs=[]
nusc=trainer.dl.dataset.helper.data
for frame_id in range(len(xshr)):
    fig, axes = plt.subplots(1, 2, figsize=(40, 20))
    anntoken=hist_tokens[frame_id]
    ann_record = nusc.get('sample_annotation', anntoken)
    sample_record = nusc.get('sample', ann_record['sample_token'])
    cam = get_cam(sample_record,nusc,anntoken)
    data_path, boxes, camera_intrinsic = nusc.get_sample_data(cam, selected_anntokens=[anntoken])
    data_path, boxes, camera_intrinsic = nusc.get_sample_data(cam, selected_anntokens=[anntoken])
    sd_record = nusc.get('sample_data', cam)
    cs_record = nusc.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
    # sensor_record = nusc.get('sensor', cs_record['sensor_token'])
    pose_record = nusc.get('ego_pose', sd_record['ego_pose_token'])
    im = Image.open(data_path)

    axes[1].imshow(im)
    axes[1].set_title('Frame '+ str(hist_frames-frame_id) + "    Status: Visible", fontsize=35)
    axes[1].axis('off')
    axes[1].set_aspect('equal')
    def get_color(name):
        return nusc.colormap[name]
    assert len(boxes)==1
    
    for box in boxes:
        c = np.array(get_color(box.name)) / 255.0
        box.render(axes[1], view=camera_intrinsic, normalize=True, colors=(c, c, c))
    
    x=xshr[frame_id]
    y=yshr[frame_id]
    dx=dxshr[frame_id]
    dy=dyshr[frame_id]
    axes[0].arrow(x, y, dx, dy, width=6.0, color=(1.        , 0.38823529, 0.27843137,1))

    for x, y, dx, dy in zip(xsl, ysl, dxsl, dysl):
        axes[0].arrow(x, y, dx, dy, width=1.0, color=(1,0.5,0,0.3))
    axes[0].imshow(image)
    axes[0].grid(False)
    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    hist_imgs.append(image_from_plot)
hist_imgs=hist_imgs[::-1]

In [ ]:
# xspr=xsp[::-1]
# yspr=ysp[::-1]
# dxspr=dxsp[::-1]
# dyspr=dysp[::-1]
xspr=xsp
yspr=ysp
dxspr=dxsp
dyspr=dysp
pred_imgs=[]
nusc=trainer.dl.dataset.helper.data
for frame_id in range(len(xspr)):
    fig, axes = plt.subplots(1, 2, figsize=(40, 20))
    anntoken=missing_frames[frame_id]['ann_token']
    ann_record = nusc.get('sample_annotation', anntoken)
    sample_record = nusc.get('sample', ann_record['sample_token'])
    cam = get_cam(sample_record,nusc,anntoken)
    data_path, boxes, camera_intrinsic = nusc.get_sample_data(cam, selected_anntokens=[anntoken])
    data_path, boxes, camera_intrinsic = nusc.get_sample_data(cam, selected_anntokens=[anntoken])
    sd_record = nusc.get('sample_data', cam)
    cs_record = nusc.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
    # sensor_record = nusc.get('sensor', cs_record['sensor_token'])
    pose_record = nusc.get('ego_pose', sd_record['ego_pose_token'])
    im = Image.open(data_path)

    axes[1].imshow(im)
    axes[1].set_title('Frame '+ str(frame_id+hist_frames+1) + "    Status: Occluded", fontsize=35)
    axes[1].axis('off')
    axes[1].set_aspect('equal')
    def get_color(name):
        return nusc.colormap[name]
    assert len(boxes)==1
    
    for box in boxes:
        box.name="human.pedestrian.adult"
        c = np.array(get_color(box.name)) / 255.0
        box.render(axes[1], view=camera_intrinsic, normalize=True, colors=(c, c, c))
    
    x=xspr[frame_id]
    y=yspr[frame_id]
    dx=dxspr[frame_id]
    dy=dyspr[frame_id]
    axes[0].arrow(x, y, dx, dy, width=6.0, color=(0.        , 0.        , 0.90196078,1))

    for x, y, dx, dy in zip(xsl, ysl, dxsl, dysl):
        axes[0].arrow(x, y, dx, dy, width=1.0, color=(1,0.5,0,0.3))
    axes[0].imshow(image)
    axes[0].grid(False)
    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    pred_imgs.append(image_from_plot)

In [ ]:
xsfr=xs[::-1]
ysfr=ys[::-1]
dxsfr=dxs[::-1]
dysfr=dys[::-1]
# xshr=xsh
# yshr=ysh
# dxshr=dxsh
# dyshr=dysh
fut_imgs=[]
nusc=trainer.dl.dataset.helper.data
for frame_id in range(fut_frames):
    fig, axes = plt.subplots(1, 2, figsize=(40, 20))
    anntoken=fut_tokens[frame_id]
    ann_record = nusc.get('sample_annotation', anntoken)
    sample_record = nusc.get('sample', ann_record['sample_token'])
    cam = get_cam(sample_record,nusc,anntoken)
    data_path, boxes, camera_intrinsic = nusc.get_sample_data(cam, selected_anntokens=[anntoken])
    data_path, boxes, camera_intrinsic = nusc.get_sample_data(cam, selected_anntokens=[anntoken])
    sd_record = nusc.get('sample_data', cam)
    cs_record = nusc.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
    # sensor_record = nusc.get('sensor', cs_record['sensor_token'])
    pose_record = nusc.get('ego_pose', sd_record['ego_pose_token'])
    im = Image.open(data_path)

    axes[1].imshow(im)
    axes[1].set_title('Frame '+ str(frame_id+len(pred_imgs)+hist_frames+1) + "    Status: Visible", fontsize=35)
    axes[1].axis('off')
    axes[1].set_aspect('equal')
    def get_color(name):
        return nusc.colormap[name]
    assert len(boxes)==1
    
    for box in boxes:
        c = np.array(get_color(box.name)) / 255.0
        box.render(axes[1], view=camera_intrinsic, normalize=True, colors=(c, c, c))
    
    x=xsfr[frame_id]
    y=ysfr[frame_id]
    dx=dxsfr[frame_id]
    dy=dysfr[frame_id]
    axes[0].arrow(x, y, dx, dy, width=6.0, color=(1.        , 0.38823529, 0.27843137,1))

    for x, y, dx, dy in zip(xsl, ysl, dxsl, dysl):
        axes[0].arrow(x, y, dx, dy, width=1.0, color=(1,0.5,0,0.3))
    axes[0].imshow(image)
    axes[0].grid(False)
    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    fut_imgs.append(image_from_plot)

In [ ]:
imgs=hist_imgs+pred_imgs+fut_imgs

In [ ]:
import imageio
filename = os.path.join('./tmp', vis_sample_token,' GIF_sample' + '.gif')
imageio.mimsave(filename, imgs, format='GIF', fps=2)

In [ ]:
import json

file_path = os.path.join('vis_data','no_point'+'.json')
with open(file_path, 'r') as json_file:
    data_list=json.load(json_file)

In [ ]:
data_dict={}
for token_dict in data_list:
    key=token_dict['start']['ins_token']+"_"+token_dict['start']['sample_token']
    data_dict[key]=token_dict

In [ ]:
history=data_test['target_agent_representation']['history']
future=data_test['target_agent_representation']['future']
concat_motion=data_test['target_agent_representation']['concat_motion']
time_query=data_test['target_agent_representation']['time_query']
refine_input=data_test['target_agent_representation']['refine_input']

In [ ]:
for filename in os.listdir('tmp'):
    if filename.endswith('.png'):
        key=filename[:-4]
        token_dict=data_dict[key]

In [ ]:
predcitions=trainer.model(data_test)
predcitions

In [ ]:
torch.sum(torch.norm(gt_test['traj'][:,:,:2],2,dim=-1),dim=-1)

In [ ]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='/home/stanliu/data/mnt/nuScenes/nuscenes', verbose=True)



In [ ]:
my_scene = nusc.scene[0]
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)


sensor = 'CAM_FRONT'
cam_front_data = nusc.get('sample_data', my_sample['data'][sensor])
cam_front_data



In [ ]:
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box


In [ ]:

my_annotation_token = my_sample['anns'][18]
my_annotation_metadata =  nusc.get('sample_annotation', my_annotation_token)


nusc.render_annotation(my_annotation_token)

